<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/07_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 檢查互相關峰值是否明顯（信號強度不夠）
if np.max(corr) < 1e-3:
    lag = 0  # 太小 → 相似性不足 → 不移動

# 2. 比最大值和平均的比值
if np.max(corr) / np.mean(np.abs(corr)) < 3:
    lag = 0

# 3. 預先過濾掉「幾乎為常數」的訊號
if np.std(sig) < 1e-3:
    lag = 0

In [ ]:
from scipy.signal import correlate
from scipy.stats import zscore
import numpy as np

def compute_alignment_offsets(signals, reference_index=0, max_lag=20, min_corr_ratio=3):
    ref = signals[reference_index]
    offsets = []

    for i, sig in enumerate(signals):
        if np.std(sig) < 1e-6:  # 幾乎是水平線，不處理
            offsets.append(0)
            continue
        ref_norm = zscore(ref)
        sig_norm = zscore(sig)
        corr = correlate(sig_norm, ref_norm, mode='full', method='fft')

        lag = np.argmax(corr) - (len(ref) - 1)

        # Normalized cross-correlation quality gating
        peak = np.max(corr)
        mean_corr = np.mean(np.abs(corr))
        ratio = peak / mean_corr if mean_corr != 0 else 0 # 避免 mean_corr=0 時爆炸

        if ratio < min_corr_ratio or abs(lag) > max_lag:
            lag = 0  # 不做對齊

        offsets.append(lag)

    return offsets


## 第幾個lag開始

In [ ]:
# 表示這條訊號應該從 第 lag 格起始位置畫起，才能與參考訊號對齊。
# (拿它作為 x 座標偏移來畫圖，不用再倒過來想誰往哪邊平移)
lag = -(np.argmax(corr) - (len(ref) - 1))
